# Initial data read
Notebook to begin exploring the UK air pollution data made available by the department for environment, food and rural affairs: [UK Air Information Resource](https://uk-air.defra.gov.uk/).  
&copy; Crown 2020 copyright Defra via uk-air.defra.gov.uk, licenced under the [Open Government Licence (OGL)](http://www.nationalarchives.gov.uk/doc/open-government-licence/version/2/).  
  
First, we will inspect air pollution readings from a measurement station in Oxford, UK.  
  
Notebook successfully:
- Achieves initial read-in of air pollution data from Oxford St Ebbes measurement station
- Point 2

In [3]:
%load_ext autoreload
%load_ext watermark
%load_ext lab_black

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark
The lab_black extension is already loaded. To reload it, use:
  %reload_ext lab_black


In [4]:
import sys
import time
from pathlib import Path
import pandas as pd
import numpy as np
import altair as alt

repo_root = Path.cwd().resolve().parent

sys.path.append(str(repo_root))
interim_data_path = repo_root / "data" / "interim"

%watermark -ntz -p pandas -a Simon-Lee-UK

Simon-Lee-UK Wed Sep 09 2020 21:08:16 BST 

pandas 1.1.2


## Data info
Location: Oxford St Ebbes (OX8)
All Data GMT hour ending  
Status: R = Ratified / P = Provisional / P* = As supplied  
Data available: 2008 - 2020
  
### Raw columns (2020):
`Date` - format: 'dd-mm-yyyy'  
`time` - 24 hour, e.g. '19:00'  
`PM<sub>10</sub> particulate matter (Hourly measured)` - $PM_{10}$ particulate matter with diameter $< 10 \;\mu m$  
`status` - ratified / provisional / as supplied  
`unit` - $\mu g \:/\: m^{3}$ (FIDAS)  
`Nitric oxide` - 'NO'  
`status.1` - ratified / provisional / as supplied  
`unit.1` - $\mu g \:/\: m^{3}$  
`Nitrogen dioxide` - 'NO$_2$'  
`status.2` - ratified / provisional / as supplied  
`unit.2` - $\mu g \:/\: m^{3}$  
`Nitrogen oxides as nitrogen dioxide` - 'NO$_x$ as NO$_2$'  
`status.3` - ratified / provisional / as supplied  
`unit.3` - $\mu g \:/\: m^{3}$  
`PM<sub>2.5</sub> particulate matter (Hourly measured)` - $PM_{2.5}$ particulate matter with diameter $< 2.5 \;\mu m$    
`status.4` - ratified / provisional / as supplied  
`unit.4` - $\mu g \:/\: m^{3}$ (Ref.eq)  
  
#### Additional columns found in data from previous years
`Volatile PM<sub>2.5</sub> (Hourly measured)` - sub-category of particulate matter measurement ($< 2.5 \;\mu m$)  
`Volatile PM2.5 (Hourly measured)` - alternate column title for the above change observed between 2010 and 2011; never both observed for same year's data; 'column_consistency' function renames this column to match the above  
`Volatile PM<sub>10</sub> (Hourly measured)` - sub-category of particulate matter measurement ($< 10 \;\mu m$)  
`Non-volatile PM<sub>2.5</sub> (Hourly measured)` - sub-category of particulate matter measurement ($< 2.5 \;\mu m$)  
`Non-volatile PM<sub>10</sub> (Hourly measured)` - sub-category of particulate matter measurement ($< 10 \;\mu m$)  
  
Volatile/Non-volatile measurements are only available in some years. The two values add to give the generic particulate matter measurement for each hour; this value is available for both sizes in all available years of data. For now, volatile/non-volatile values will be dropped because they cannot be compared across the full range of data.

## Data read functions
Below, define functions for reading and pre-processing the raw data. These functions will later be exported out to a separate package for ease of maintenance and for access from future notebooks.

In [2]:
def get_single_year_raw(location_prefix, year):
    data_url = (
        f"https://uk-air.defra.gov.uk/data_files/site_data/{location_prefix}_{year}.csv"
    )
    single_year = pd.read_csv(data_url, header=4)
    return single_year


def column_consistency(raw_data):
    column_consist_dict = {
        "Volatile PM2.5 (Hourly measured)": "Volatile PM<sub>2.5</sub> (Hourly measured)"
    }

    consistent_columns = raw_data.rename(columns=column_consist_dict)
    return consistent_columns


def column_conversion(raw_data):
    column_dict = {
        "Date": "date",
        "PM<sub>10</sub> particulate matter (Hourly measured)": "pm_10",
        "status": "status_pm_10",
        "unit": "unit_pm_10",
        "Nitric oxide": "nitric_oxide",
        "status.1": "status_nitric_oxide",
        "unit.1": "unit_nitric_oxide",
        "Nitrogen dioxide": "nitrogen_dioxide",
        "status.2": "status_nitrogen_dioxide",
        "unit.2": "unit_nitrogen_dioxide",
        "Nitrogen oxides as nitrogen dioxide": "NO2_eq",
        "status.3": "status_NO2_eq",
        "unit.3": "unit_NO2_eq",
        "PM<sub>2.5</sub> particulate matter (Hourly measured)": "pm_2_5",
        "status.4": "status_pm_2_5",
        "unit.4": "unit_pm_2_5",
    }

    converted_columns = raw_data.rename(columns=column_dict)
    converted_columns = extend_date_with_time(converted_columns)
    return converted_columns


def extend_date_with_time(raw_data):
    extended_date = raw_data.copy()
    extended_date.time = extended_date.time.replace("24:00", "00:00")

    extended_date = datetime_conversion(extended_date, date_format="%d-%m-%Y")
    update_mask = extended_date.time == "00:00"
    extended_date.loc[update_mask, "date"] = extended_date.loc[
        update_mask, "date"
    ] + pd.to_timedelta(1, unit="d")
    extended_date.date = extended_date.date.astype("str")

    extended_date.date = extended_date.date + " " + extended_date.time
    return extended_date


def datetime_conversion(raw_data, target_column="date", date_format="%Y-%m-%d %H:%M"):
    converted_datetime = raw_data.copy()
    try:
        converted_datetime[target_column] = pd.to_datetime(
            converted_datetime[target_column], format=date_format
        )
    except ValueError:
        print(
            "date_format in 'datetime_conversion' function could not be applied generally"
        )
        converted_datetime[target_column] = pd.to_datetime(
            converted_datetime[target_column]
        )

    return converted_datetime

## Combining multiple years of data
Data is available for years 2008 - 2020 with access URLs such as: 'https://uk-air.defra.gov.uk/data_files/site_data/OX8_2008.csv' providing the data for each individual year.  

Use this consistent csv file name format to loop through the available years and save a composite csv file containing all available data.  
Start by defining the list of years we want to download data for:

In [3]:
location = "OX8"
start_year = 2008
end_year = 2020
years_of_interest = list(np.arange(start_year, end_year + 1))

### Inconsistent column titles between data sets
Following initial read in of some test years of data, some column titles were seen for certain years of data but not others. The full set of observed columns (ignoring associated 'status' and 'unit' columns) was copied to the DataFrame defined in the cell below. This DataFrame is used later to record the presence/absence of each column for each available year of air pollution data. 

In [4]:
inspect_columns = pd.DataFrame(
    {
        "Data (Year)": ["blank"],
        "Date": [False],
        "time": [False],
        "Nitric oxide": [False],
        "Nitrogen dioxide": [False],
        "Nitrogen oxides as nitrogen dioxide": [False],
        "PM<sub>10</sub> particulate matter (Hourly measured)": [False],
        "PM<sub>2.5</sub> particulate matter (Hourly measured)": [False],
        "Volatile PM<sub>10</sub> (Hourly measured)": [False],
        "Volatile PM<sub>2.5</sub> (Hourly measured)": [False],
        "Non-volatile PM<sub>10</sub> (Hourly measured)": [False],
        "Non-volatile PM<sub>2.5</sub> (Hourly measured)": [False],
    }
)
inspect_columns = inspect_columns.append(
    [inspect_columns] * (len(years_of_interest) - 1), ignore_index=True
)

### Looping to read data for years of interest
We also fill the DataFrame defined in the cell above based on the presence/absence of each possible column for each individual year of data after it is read from the source URL.

In [5]:
query_columns = inspect_columns.columns.tolist()

for idx, indv_year in enumerate(years_of_interest):
    single_year = get_single_year_raw(location, year=indv_year)
    single_year = column_consistency(single_year)

    single_year_cols = single_year.columns.tolist()
    inspect_columns.loc[idx, "Data (Year)"] = indv_year
    for col in query_columns:
        if col in single_year_cols:
            inspect_columns.loc[idx, col] = True

    processed_year = single_year.pipe(column_conversion).pipe(datetime_conversion)
    if idx == 0:
        air_pollution = processed_year.copy()
    else:
        air_pollution = air_pollution.append(processed_year, ignore_index=True)
    time.sleep(1.5)  # creates interval between requests to uk-air.defra.gov.uk

Inspect which columns appear in which years' data sets:

In [6]:
inspect_columns

,Data (Year),Date,time,Nitric oxide,Nitrogen dioxide,Nitrogen oxides as nitrogen dioxide,PM<sub>10</sub> particulate matter (Hourly measured),PM<sub>2.5</sub> particulate matter (Hourly measured),Volatile PM<sub>10</sub> (Hourly measured),Volatile PM<sub>2.5</sub> (Hourly measured),Non-volatile PM<sub>10</sub> (Hourly measured),Non-volatile PM<sub>2.5</sub> (Hourly measured)
0,2008,True,True,True,True,True,True,True,False,True,False,True
1,2009,True,True,True,True,True,True,True,True,True,True,True
2,2010,True,True,True,True,True,True,True,True,True,True,True
3,2011,True,True,True,True,True,True,True,True,True,True,True
4,2012,True,True,True,True,True,True,True,True,True,True,True
5,2013,True,True,True,True,True,True,True,True,True,True,True
6,2014,True,True,True,True,True,True,True,True,True,True,True
7,2015,True,True,True,True,True,True,True,True,True,True,True
8,2016,True,True,True,True,True,True,True,True,True,True,True
9,2017,True,True,True,True,True,True,True,True,True,True,True


Where years have both volatile and non-volatile particulate matter readings: the sum of these two values corresponds to the value in the 'PM<sub>xx</sub>' column (see ['compare particulate matter readings'](#compare-particulate-matter-readings) section below). Happy to just drop these for now (as they're not included in all years). Use the cell below to extract those columns that are to be dropped (data not available for all years)

In [7]:
common_columns = pd.Series(inspect_columns.all())
drop_columns = common_columns.copy()
drop_columns = drop_columns[~drop_columns]
drop_columns = drop_columns.index.tolist()

volatile = (
    air_pollution.copy()
)  # define new DataFrame for section below before columns are dropped

air_pollution.drop(drop_columns, axis="columns", inplace=True)

### Compare particulate matter readings
Check to ensure that the volatile/non-volatile particulate matter readings sum to give the associated generic value for both 2.5 and 10 $\mu m$ sizes.  
Start by defining a new subset of the data looking only at the columns of interest (for a small portion of the data):

In [8]:
volatile = volatile[
    [
        "date",
        "Non-volatile PM<sub>2.5</sub> (Hourly measured)",
        "Volatile PM<sub>2.5</sub> (Hourly measured)",
        "pm_2_5",
        "Non-volatile PM<sub>10</sub> (Hourly measured)",
        "Volatile PM<sub>10</sub> (Hourly measured)",
        "pm_10",
    ]
]
# sample_size = 5
# volatile = volatile.sample(sample_size, random_state=5)

For this sample data, define two new columns that sum the values in the volatile and non-volatile column for each size. Then, compare the values in this column to those in the generic column for each row:

In [9]:
volatile["sum_2_5"] = (
    volatile["Non-volatile PM<sub>2.5</sub> (Hourly measured)"]
    + volatile["Volatile PM<sub>2.5</sub> (Hourly measured)"]
).round(1)
volatile["sum_10"] = (
    volatile["Non-volatile PM<sub>10</sub> (Hourly measured)"]
    + volatile["Volatile PM<sub>10</sub> (Hourly measured)"]
).round(1)
volatile.reset_index(drop=True, inplace=True)
volatile

,date,Non-volatile PM<sub>2.5</sub> (Hourly measured),Volatile PM<sub>2.5</sub> (Hourly measured),pm_2_5,Non-volatile PM<sub>10</sub> (Hourly measured),Volatile PM<sub>10</sub> (Hourly measured),pm_10,sum_2_5,sum_10
0,2008-01-01 01:00:00,NaN,NaN,NaN,NaN,NaN,29.0,NaN,NaN
1,2008-01-01 02:00:00,NaN,NaN,NaN,NaN,NaN,29.0,NaN,NaN
2,2008-01-01 03:00:00,NaN,NaN,NaN,NaN,NaN,19.0,NaN,NaN
3,2008-01-01 04:00:00,NaN,NaN,NaN,NaN,NaN,16.0,NaN,NaN
4,2008-01-01 05:00:00,NaN,NaN,NaN,NaN,NaN,12.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...
107443,2020-04-03 20:00:00,NaN,NaN,5.660,NaN,NaN,9.3,NaN,NaN
107444,2020-04-03 21:00:00,NaN,NaN,8.396,NaN,NaN,12.1,NaN,NaN
107445,2020-04-03 22:00:00,NaN,NaN,12.925,NaN,NaN,16.9,NaN,NaN
107446,2020-04-03 23:00:00,NaN,NaN,13.208,NaN,NaN,17.7,NaN,NaN


Cell below prints df for those particulate matter values don't add up as we expect:

In [10]:
check_equal = volatile.copy()
equal_mask = (
    (check_equal["pm_2_5"] == check_equal["sum_2_5"])
    | (
        (check_equal["Non-volatile PM<sub>2.5</sub> (Hourly measured)"].isna())
        & (check_equal["Volatile PM<sub>2.5</sub> (Hourly measured)"].isna())
    )
) & (
    (check_equal["pm_10"] == check_equal["sum_10"])
    | (
        (check_equal["Non-volatile PM<sub>10</sub> (Hourly measured)"].isna())
        & (check_equal["Volatile PM<sub>10</sub> (Hourly measured)"].isna())
    )
)

conflicts = check_equal.copy()
conflicts = conflicts[~equal_mask]

In [11]:
conflicts

,date,Non-volatile PM<sub>2.5</sub> (Hourly measured),Volatile PM<sub>2.5</sub> (Hourly measured),pm_2_5,Non-volatile PM<sub>10</sub> (Hourly measured),Volatile PM<sub>10</sub> (Hourly measured),pm_10,sum_2_5,sum_10
95304,2018-11-15 01:00:00,11.2,6.8,14.5,16.3,3.3,19.6,18.0,19.6
95305,2018-11-15 02:00:00,15.8,2.6,14.9,17.3,4.4,21.7,18.4,21.7
95306,2018-11-15 03:00:00,13.5,7.9,17.9,17.2,4.8,22.0,21.4,22.0
95307,2018-11-15 04:00:00,15.6,3.0,15.1,15.0,4.4,19.4,18.6,19.4
95308,2018-11-15 05:00:00,9.3,7.2,13.0,13.4,3.8,17.2,16.5,17.2
...,...,...,...,...,...,...,...,...,...
97172,2019-01-31 21:00:00,NaN,NaN,NaN,16.5,1.9,13.1,NaN,18.4
97173,2019-01-31 22:00:00,NaN,NaN,NaN,19.0,2.1,15.8,NaN,21.1
97174,2019-01-31 23:00:00,NaN,NaN,NaN,14.5,4.4,13.6,NaN,18.9
97175,2019-02-01 00:00:00,NaN,NaN,NaN,10.5,5.3,10.5,NaN,15.8


In [12]:
percentage_conflicts = (len(conflicts) / len(volatile)) * 100
print(
    f"{percentage_conflicts} % of data shows difference between calculated and raw data values of general particulate matter measurements"
)

1.616595934777753 % of data shows difference between calculated and raw data values of general particulate matter measurements


One 'ToDo' is to expand the above check into a general check against the full final-state DataFrame...

## Save composite data file

In [13]:
if not interim_data_path.is_dir():
    interim_data_path.mkdir(parents=True)

save_location = interim_data_path / "oxford_ebbes_full.csv"
air_pollution.to_csv(save_location, index=False)
print(f"Full historic air pollution data saved to: {save_location}")

Full historic air pollution data saved to: /Users/Simon/Documents/Python_Projects/Productivity Boiz/air-pollution/data/interim/oxford_ebbes_full.csv
